# Exploring movie ratings dataset with Spark SQL

This project focuses on exploring the MovieLens dataset using Spark SQL, which allows us to execute SQL queries on distributed datasets.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [2]:
# Create a SparkSession as entry point to Spark SQL functionality and is required to work with dataframes and perform SQL queries
spark = SparkSession.builder.appName("SparkSQLExploration").getOrCreate()

In [3]:
# Load the MovieLens dataset into a dataframe and define column names
data_df = spark.read.option("delimiter", "\t").csv("ml-100k/ml-100k/u.data").toDF("user_id", "movie_id", "rating", "timestamp")
# Register dataframe as a temporary view. This allows us to query the dataframe using SQL syntax by creating a temporary view called "ratings"
data_df.createOrReplaceTempView("ratings")

In [4]:
# Count the number of ratings by executing a SQL query on the "ratings" temporary view
# The collect method retrieves the result as a list of row objects. We access the first element of the list and then the value of the "num_ratings" column
num_ratings = spark.sql("SELECT COUNT(*) AS num_ratings FROM ratings").collect()[0]["num_ratings"]
print(f"Number of ratings: {num_ratings}")

Number of ratings: 100000


In [5]:
# Calculate the total rating and count per movie using SQL
movie_rating_counts = spark.sql("""
    SELECT movie_id, SUM(rating) AS total_rating, COUNT(*) AS rating_count
    FROM ratings
    GROUP BY movie_id
""")
# The result is stored in a new dataframe called movie_rating_counts to allow subsequent SQL queries to reference this view
movie_rating_counts.createOrReplaceTempView("movie_rating_counts")

# Calculate the average rating per movie
average_rating_per_movie = spark.sql("""
    SELECT movie_id, total_rating / rating_count AS avg_rating
    FROM movie_rating_counts
""")
print("Average rating per movie:")
average_rating_per_movie.show(5)

Average rating per movie:
+--------+------------------+
|movie_id|        avg_rating|
+--------+------------------+
|     829|2.6470588235294117|
|    1436|               2.5|
|     467|3.7916666666666665|
|     691|               3.5|
|    1090|2.4054054054054053|
+--------+------------------+
only showing top 5 rows



In [6]:
# Calculate the total number of ratings per movie
total_ratings_per_movie = spark.sql("""
    SELECT movie_id, rating_count
    FROM movie_rating_counts
""")
print("Total ratings per movie:")
total_ratings_per_movie.show(5)

# Filter movies with a minimum number of ratings
popular_movies = spark.sql("""
    SELECT *
    FROM movie_rating_counts
    WHERE rating_count >= 100
""")
print("\nPopular movies:")
popular_movies.show(5)

Total ratings per movie:
+--------+------------+
|movie_id|rating_count|
+--------+------------+
|     829|          34|
|    1436|           2|
|     467|          48|
|     691|          16|
|    1090|          37|
+--------+------------+
only showing top 5 rows


Popular movies:
+--------+------------+------------+
|movie_id|total_rating|rating_count|
+--------+------------+------------+
|     451|       569.0|         170|
|     125|       868.0|         244|
|     926|       273.0|         101|
|     591|       641.0|         178|
|       7|      1489.0|         392|
+--------+------------+------------+
only showing top 5 rows



In [7]:
# Find the most and least rated movies
most_rated_movies = spark.sql("""
    SELECT movie_id, rating_count
    FROM movie_rating_counts
    ORDER BY rating_count DESC
""")
print("Most rated movies:")
most_rated_movies.show(5)

least_rated_movies = spark.sql("""
    SELECT movie_id, rating_count
    FROM movie_rating_counts
    ORDER BY rating_count
""")
print("\nLeast rated movies:")
least_rated_movies.show(5)

Most rated movies:
+--------+------------+
|movie_id|rating_count|
+--------+------------+
|      50|         583|
|     258|         509|
|     100|         508|
|     181|         507|
|     294|         485|
+--------+------------+
only showing top 5 rows


Least rated movies:
+--------+------------+
|movie_id|rating_count|
+--------+------------+
|    1349|           1|
|    1601|           1|
|    1572|           1|
|    1492|           1|
|    1669|           1|
+--------+------------+
only showing top 5 rows



In [8]:
# Find users who rated the most movies
most_active_users = spark.sql("""
    SELECT user_id, COUNT(*) AS num_ratings
    FROM ratings
    GROUP BY user_id
    ORDER BY num_ratings DESC
""")
print("Users who rated the most movies:")
most_active_users.show(5)

Users who rated the most movies:
+-------+-----------+
|user_id|num_ratings|
+-------+-----------+
|    405|        737|
|    655|        685|
|     13|        636|
|    450|        540|
|    276|        518|
+-------+-----------+
only showing top 5 rows



In [9]:
# Find the distribution of ratings and sort by key
rating_distribution = spark.sql("""
    SELECT rating, COUNT(*) AS count
    FROM ratings
    GROUP BY rating
    ORDER BY rating
""")
print("Rating distribution:")
rating_distribution.show()

Rating distribution:
+------+-----+
|rating|count|
+------+-----+
|     1| 6110|
|     2|11370|
|     3|27145|
|     4|34174|
|     5|21201|
+------+-----+



In [10]:
# Group ratings by movie id and calculate sum and count of ratings for each movie
movie_rating_sum_count = spark.sql("""
    SELECT movie_id, SUM(rating) AS total_rating, COUNT(*) AS rating_count
    FROM ratings
    GROUP BY movie_id
""")

print("\Sum and count of rating per movie:")
# Setting truncate to false ensures that the full content of each column is displayed without truncation
movie_rating_sum_count.show(5, truncate=False)

# Calculate average rating for each movie
average_rating_per_movie = spark.sql("""
    SELECT movie_id, total_rating / rating_count AS avg_rating
    FROM (
        SELECT movie_id, SUM(rating) AS total_rating, COUNT(*) AS rating_count
        FROM ratings
        GROUP BY movie_id
    )
""")
# Register DataFrame as a temporary view
average_rating_per_movie.createOrReplaceTempView("average_rating_per_movie")

print("\nAverage rating per movie:")
average_rating_per_movie.show(5, truncate=False)

\Sum and count of rating per movie:
+--------+------------+------------+
|movie_id|total_rating|rating_count|
+--------+------------+------------+
|829     |90.0        |34          |
|1436    |5.0         |2           |
|467     |182.0       |48          |
|691     |56.0        |16          |
|1090    |89.0        |37          |
+--------+------------+------------+
only showing top 5 rows


Average rating per movie:
+--------+------------------+
|movie_id|avg_rating        |
+--------+------------------+
|829     |2.6470588235294117|
|1436    |2.5               |
|467     |3.7916666666666665|
|691     |3.5               |
|1090    |2.4054054054054053|
+--------+------------------+
only showing top 5 rows



In [11]:
# Read the CSV file of movies, select relevant columns
movie_titles_genres_df = spark.read.option("delimiter", "|").csv("ml-100k/ml-100k/u.item").selectExpr("_c0 as movie_id", "_c1 as title", "_c5 as genres")
# Register DataFrame as a temporary view
movie_titles_genres_df.createOrReplaceTempView("movies")

print("Values of movies:")
movie_titles_genres_df.show(5)

# Join movie_titles_genres with average_rating_per_movie by movie_id
movies_with_ratings = spark.sql("""
    SELECT m.movie_id, m.title, m.genres, a.avg_rating
    FROM movies m
    JOIN average_rating_per_movie a
    ON m.movie_id = a.movie_id
""")

print("\nMovies with averaged rating:")
movies_with_ratings.show(5)

Values of movies:
+--------+-----------------+------+
|movie_id|            title|genres|
+--------+-----------------+------+
|       1| Toy Story (1995)|     0|
|       2| GoldenEye (1995)|     0|
|       3|Four Rooms (1995)|     0|
|       4|Get Shorty (1995)|     0|
|       5|   Copycat (1995)|     0|
+--------+-----------------+------+
only showing top 5 rows


Movies with averaged rating:
+--------+--------------------+------+------------------+
|movie_id|               title|genres|        avg_rating|
+--------+--------------------+------+------------------+
|     829|         Fled (1996)|     0|2.6470588235294117|
|    1436|    Mr. Jones (1993)|     0|               2.5|
|     467|Bronx Tale, A (1993)|     0|3.7916666666666665|
|     691|    Dark City (1998)|     0|               3.5|
|    1090|       Sliver (1993)|     0|2.4054054054054053|
+--------+--------------------+------+------------------+
only showing top 5 rows



In [12]:
# Extract genres and count distinct values
distinct_genres_count = spark.sql("""
    SELECT COUNT(DISTINCT genre) AS distinct_genres_count
    FROM (
        SELECT explode(split(genres, ',')) AS genre
        FROM movies
    )
""").collect()[0]["distinct_genres_count"]
print(f"Number of genres: {distinct_genres_count}")

# Calculate average rating per genre
# First, explode genres into separate rows, then join with ratings and calculate average rating per genre
average_rating_per_genre_df = spark.sql("""
    SELECT genre, AVG(avg_rating) AS avg_rating
    FROM (
        SELECT explode(split(genres, ',')) AS genre, a.avg_rating
        FROM movies m
        JOIN average_rating_per_movie a ON m.movie_id = a.movie_id
    )
    GROUP BY genre
""")

print("\nAverage rating per genre:")
average_rating_per_genre_df.show(5)

Number of genres: 2

Average rating per genre:
+-----+------------------+
|genre|        avg_rating|
+-----+------------------+
|    0|3.0770609634276833|
|    1|2.2222222222222223|
+-----+------------------+



In [13]:
# Define user defined function (udf), for example, convert rating to sentiment
def rating_sentiment(rating):
    rating = int(rating)
    if rating >= 4:
        return "Positive"
    elif rating <= 2:
        return "Negative"
    else:
        return "Neutral"

# Register UDFs with Spark SQL. The udf.register method allows us to register a python function as a UDF that can be used in Spark SQL queries
# The first argument is the name of the UDF that we will use in SQL queries. The third argument specifies the return type of the UDF
spark.udf.register("rating_sentiment", rating_sentiment, StringType())

# Calculate sentiment distribution of ratings
rating_sentiment_distribution = spark.sql("""
    SELECT rating_sentiment(rating) AS sentiment, COUNT(*) AS count
    FROM ratings
    GROUP BY rating_sentiment(rating)
""")

print("Sentiment distribution of ratings:")
rating_sentiment_distribution.show()

Sentiment distribution of ratings:
+---------+-----+
|sentiment|count|
+---------+-----+
|  Neutral|27145|
| Positive|55375|
| Negative|17480|
+---------+-----+



In [14]:
# Stop SparkSession
spark.stop()